# Install libraires

In [ ]:
import pandas as pd
import os
import nltk
import torchvision
import torch
nltk.download('punkt')
from google.colab import files
from google.colab import drive
!pip install chamd
from matplotlib import pyplot
from numpy import where
from chamd import ChatReader
import numpy as np
from collections import defaultdict
from collections import Counter
from tokenizers import BertWordPieceTokenizer
# from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import BertTokenizer, BertForSequenceClassification
!pip install transformers
from transformers import DistilBertTokenizer, BertModel
from torch.utils.data import DataLoader
from transformers import TFAutoModelForSequenceClassification

from transformers import create_optimizer
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback
from transformers import TFBertForSequenceClassification
import torch.nn as nn
!pip install evaluate
import evaluate


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Data Preprocessing
## Deal with first dataset



In [ ]:
# variables for controlling the number of samples in dataset
# reduce some samples to shorten the data preprocessing time
n = 20000

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
hatespeechfile = '/content/drive/MyDrive/project/data/hate_speech/train.csv'
hatespeech_df = pd.read_csv(hatespeechfile)
hatespeech_df = hatespeech_df.iloc[:n,4:]
hatespeech_df

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
19995,1,RT @semper_fiibitch: Farra from teen mom has 2...
19996,2,RT @sepinwall: Day 2 of FXX&#8217;s Simpsons m...
19997,1,"RT @shabbashone ""@ThoughtsOfRandy: I almost ra..."
19998,1,RT @shaelynspacyyy: &#8220;@UglyAssDerrick: @s...


In [ ]:
# change the name of 'class' into 'immoral text'
hatespeech_df.rename(columns={'class':'immoral text'},inplace= True)
#Noted that if immoral text = 1,2 then not suitable for children, if 0, then may be suitable for children to read
hatespeech_df

,immoral text,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
19995,1,RT @semper_fiibitch: Farra from teen mom has 2...
19996,2,RT @sepinwall: Day 2 of FXX&#8217;s Simpsons m...
19997,1,"RT @shabbashone ""@ThoughtsOfRandy: I almost ra..."
19998,1,RT @shaelynspacyyy: &#8220;@UglyAssDerrick: @s...


## Deal with the second dataset to merge it with the first one

In [ ]:


hatespeechfile2 = '/content/drive/MyDrive/project/data/hate_speech(mainly not hate)/annotations_metadata.csv'
hatespeechfile2_df = pd.read_csv(hatespeechfile2)
hatespeechfile2_df = hatespeechfile2_df[['label','file_id']]
hatespeechfile2_df.replace({'noHate':0}, inplace=True, limit=None, regex=False, method='pad')
hatespeechfile2_df.replace({'hate':1}, inplace=True, limit=None, regex=False, method='pad')
hatespeechfile2_df.rename(columns = {'label':'immoral text'},inplace= True)

corpus = []
for filename in hatespeechfile2_df['file_id']:
  f = open('/content/drive/MyDrive/project/data/hate_speech(mainly not hate)/all_files/'+filename+".txt", "r")
  corpus.append(f.read())
  f.close()



hatespeechfile2_df['tweet'] = corpus
hatespeechfile2_df

In [ ]:
hatespeechfile2_df.drop('file_id',inplace=True,axis=1)


In [ ]:
hatespeech_df = pd.concat([hatespeech_df,hatespeechfile2_df],axis=0,ignore_index=True)
hatespeech_df

In [ ]:
#hatespeech_df.to_csv('/content/drive/MyDrive/project/data/hate_speech/final.csv')
hatespeech_df =pd.read_csv('/content/drive/MyDrive/project/data/hate_speech/final.csv', index_col=0)
hatespeech_df = hatespeech_df.loc[hatespeech_df['immoral text'] != 'idk/skip']
hatespeech_df = hatespeech_df.loc[hatespeech_df['immoral text'] != 'relation']
hatespeech_df['immoral text'] = hatespeech_df['immoral text'].astype(int)
hatespeech_df.reset_index(drop=True, inplace=True)
hatespeech_df
# hatespeech_df

,immoral text,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
30698,0,"Billy - `` That guy would n't leave me alone ,..."
30699,0,Wish we at least had a Marine Le Pen to vote f...
30700,0,Its like the choices are white genocide candid...
30701,1,Why White people used to say that sex was a si...


## Cleaning tweets

In [ ]:
def clean_up(input_str):
    output_str = ""
    dirt_start = -1
    user_start = -1
    url_start = -1
    prev = 0
    for i in range(len(input_str)-1):
        if (input_str[i] in [" ", ":", ";", ",", ".", "!", "?", "\n", "\t"]):
            if (url_start != -1 and input_str[i] in [":", "."]):
                continue
            if dirt_start!=-1 or user_start != -1 or url_start != -1:
                prev = i+1
                dirt_start = -1
                user_start = -1
                url_start = -1

        if dirt_start == -1 and user_start == -1 and url_start == -1:
            if input_str[i] == "&" or input_str[i] == "#":
                dirt_start = i
            elif input_str[i] == "@":
                user_start = i
            elif input_str[i:i+4] == "http":
                url_start = i
            elif input_str[i:i+2] == "RT" or input_str[i:i+2] == "!!":
                output_str += input_str[prev:i]
                prev = i+2
            else:
                continue
            output_str += input_str[prev:i]

    if dirt_start!=-1 or user_start != -1 or url_start != -1:
        prev = len(input_str)
    output_str += input_str[prev:]
    return output_str.strip()

In [ ]:
hatespeech_df['tweet'] = hatespeech_df['tweet'].apply(clean_up)
hatespeech_df

,immoral text,tweet
0,2,As a woman you shouldn't complain about cleani...
1,1,boy dats cold...tyga dwn bad for cuffin dat ho...
2,1,Dawg You ever fuck a bitch and she start to ...
3,1,she look like a tranny
4,1,The shit you hear about me might be true or it...
...,...,...
30698,0,"Billy - `` That guy would n't leave me alone ,..."
30699,0,Wish we at least had a Marine Le Pen to vote f...
30700,0,Its like the choices are white genocide candid...
30701,1,Why White people used to say that sex was a si...


## Feature Engineering: Readability

1. Tokenize using nltk function

2. Clean the unnecessary characters in the tweet

3. Count the overlapping words of the tweet and the child_corpus

4. Calculate the percentage


In [ ]:
# Deal with the new feature Readibility
# These files were generated by CLAN, a specialized program for transcribing children's speech.
# Under the hood, however, they're just text files with some additional formatting.
# With a little text processing we can just treat them like raw text files.


In [ ]:
# here would be the multiple files in cha format
child = ['/content/drive/MyDrive/project/data/children speech corpus/bnfs1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/brnd1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/chrs1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/cndx1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/dnln1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/dnns1.cha'
,  '/content/drive/MyDrive/project/data/children speech corpus/dnnc1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/dvdc1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/flpp1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/gsyn1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/jnnh1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/lgkr1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/llkc1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/mrss1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/rmlm1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/rndl1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/rndl1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/sbst1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/shhn1.cha'
, "/content/drive/MyDrive/project/data/children speech corpus/smns1.cha"
, "/content/drive/MyDrive/project/data/children speech corpus/thrj1.cha"
, '/content/drive/MyDrive/project/data/children speech corpus/tnyn1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/trrk1.cha'
, '/content/drive/MyDrive/project/data/children speech corpus/yssf1.cha']


In [ ]:
def vocab_cha_file(child):
  #child : a list storing all the file path of the cha files
  # return vocab, a list storing all the unique tokens from the child speech corpus
  size = len(child)
  reader = ChatReader()
  chat = [reader.read_file(child[i]) for i in range(size) ]

  child_corpus = [ ]
  for i in range(size):
    for line in chat[i].lines:
        child_corpus.append(line.text)


  vocab = set()
  for line in child_corpus:
    tokens = nltk.tokenize.word_tokenize(line, language='english', preserve_line=False)
    for token in tokens:
      vocab.add(token)
  return vocab


In [ ]:
child_speech_vocab = vocab_cha_file(child)
child_speech_vocab

{"d'you",
 'thank',
 'puppets',
 'Easter',
 'tape',
 'spot',
 'tennis',
 'nother',
 'eh',
 'wake+d',
 'Alexandra',
 'long',
 'Stephanie',
 'soft',
 'small',
 'need',
 'Braden',
 'Tyson',
 'Guatemala',
 'they',
 'Leila',
 'size',
 'art',
 'Beyblades',
 'Arma',
 'sitting',
 'build',
 'buffalo',
 'pearl',
 'stand',
 'Ukranian',
 'rainy',
 'dvd',
 'rubber',
 'clown',
 'hang',
 'Kaydin',
 'DaffyDuck',
 'missed+ed',
 'ouch',
 'Africa',
 'piece',
 'poor',
 "'ll",
 'thousand',
 'Lindsay',
 'El',
 'hiding',
 'buddy',
 'date',
 'farmer',
 'Hafiz',
 'bison',
 'number',
 'nintendo',
 'ordered',
 'Juananita',
 'everything',
 'strawberries',
 'dead',
 'mouse+s',
 'buried',
 'oop',
 'laid',
 'bombs',
 'crack',
 'I',
 'coloured',
 'barn',
 'swallow',
 'flown',
 'silver',
 'names',
 'chore',
 'hate',
 'Balls',
 'safety',
 'pants',
 'dumengasvil',
 'shut',
 'ho',
 'phone',
 'wrist',
 'float',
 'decided',
 'tires',
 'Diem',
 'sings',
 'flying',
 'backwards',
 'sevens',
 'talking',
 'theatre',
 'last',
 '

In [ ]:
# get readability
def readability_tweet(dataframe,vocab):
  readabilities = []
  for i in range(dataframe.shape[0]):
    tweet_tokens = nltk.tokenize.word_tokenize(dataframe['tweet'][i], language='english', preserve_line=False)
    total_words = len(tweet_tokens)
    count = 0
    for i in tweet_tokens:
       if i in vocab: count = count + 1
    if count == 0 :
      readability = 0
    else :
      readability = count / total_words
    readabilities.append(readability)
  return readabilities

In [ ]:
readabilities = readability_tweet(hatespeech_df,child_speech_vocab)

## Relabelling

In [ ]:
def convert_column_data(dataframe,columnname, condition_col):
# columnname : string, the target colunm name
# condition col: a list storing the condition columns
# return a relabelled dataframe
  condition = []
  for i in condition_col:
    temp = np.array(dataframe[i])
    if i == 'readability score': condition.append(temp >= 0.9 )
    elif i == 'immoral text': condition.append(temp == 0)
  final_condition = (condition[0].astype(int) * condition[1].astype(int))
  dataframe[columnname] = final_condition
  return dataframe

In [ ]:
hatespeech_df['readability score'] = readabilities
condition_col = ['readability score','immoral text']
hatespeech_df = convert_column_data(hatespeech_df,'Class',condition_col)
hatespeech_df

,immoral text,tweet,readability score,Class
0,2,As a woman you shouldn't complain about cleani...,0.791667,0
1,1,boy dats cold...tyga dwn bad for cuffin dat ho...,0.533333,0
2,1,Dawg You ever fuck a bitch and she start to ...,0.647059,0
3,1,she look like a tranny,0.800000,0
4,1,The shit you hear about me might be true or it...,0.772727,0
...,...,...,...,...
30698,0,"Billy - `` That guy would n't leave me alone ,...",0.650000,0
30699,0,Wish we at least had a Marine Le Pen to vote f...,0.666667,0
30700,0,Its like the choices are white genocide candid...,0.500000,0
30701,1,Why White people used to say that sex was a si...,0.800000,0


## Undersampling
We need to deal with the extremely imbalanced dataset

In [ ]:
def balance_dataset(df):

  df1 = df[df['Class'] == 0]
  df11 = df1[df1['readability score'] < 0.9]
  df12 = df1[df1['readability score'] >= 0.9]

  df2 = df[df['Class'] == 1]
  df_new = pd.concat([df11.iloc[:358,:],df12.iloc[:359]],axis=0)
  df_new = pd.concat([df_new,df2],axis =0)
  df_new = df_new.sample(frac = 1)
  return df_new

df = balance_dataset(hatespeech_df)
df.reset_index(drop=True, inplace=True)
df

,immoral text,tweet,readability score,Class
0,0,"I must say , the younger guy sure keeps this g...",0.933333,1
1,0,"Big talk , none of you have the balls to tell ...",0.944444,1
2,1,""" Andrewbryant: Poff has double vision after 2...",0.294118,0
3,1,""" Dese hoes be LYIN to all of us nigga""",0.454545,0
4,0,I own this game and it is by far the most viol...,1.000000,1
...,...,...,...,...
1429,1,the witches took them hoe,1.000000,0
1430,1,""" these hoes like niggas that spend money not ...",0.538462,0
1431,0,Those who do n't believe are free to walk away...,0.913043,1
1432,1,""" Fuck Red Malone man bitch ass niggah"" could ...",0.375000,0


In [ ]:
df.to_csv('/content/drive/MyDrive/project/data/hate_speech/df.csv')

## Train Test splitting

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/project/data/hate_speech/df.csv')
train = df.iloc[:1147]
test = df.iloc[1147:]

# BERT-based model training and testing

In [ ]:

# Define the training parameters
batch_size = [16,32]
num_epochs = [3,5,7,9]
learning_rate = [1e-5,2e-5,3e-5,5e-5]



def train_model(batch_size,num_epochs,lr):
    # Train the model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2, output_attentions=False, output_hidden_states=False)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    encoded_data = tokenizer.batch_encode_plus(train['tweet'].values, add_special_tokens=True, return_attention_mask=True, padding=True, return_tensors='pt',truncation=True)
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(train['Class'].values)

    for epoch in range(num_epochs):
        total_loss = 0
        model.train()
        for i in range(0, input_ids.size(0), batch_size):
            optimizer.zero_grad()
            outputs = model(input_ids[i:i+batch_size], attention_mask=attention_masks[i:i+batch_size], labels=labels[i:i+batch_size])
            # Cross Entropy Loss
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}')

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_masks).logits
        predictions = torch.argmax(logits, dim=1).flatten()
        final_accuracy = torch.sum(predictions == labels) / len(labels)

    print("Accuracy:", final_accuracy.item())
    return model,final_accuracy,total_loss

In [ ]:
def test_model(model,test,batch_size):
    accuracys = []
    f1s = []
    precisions = []
    recalls = []
    c_matrix = np.zeros((2,2)).astype(int)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    encoded_data = tokenizer.batch_encode_plus(test['tweet'].values, add_special_tokens=True, return_attention_mask=True, padding=True, return_tensors='pt',truncation=True)
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    labels = torch.tensor(test['Class'].values)
    metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    total_loss = 0
    for i in range(0, input_ids.size(0), batch_size):
      outputs = model(input_ids[i:i+batch_size], attention_mask=attention_masks[i:i+batch_size], labels=labels[i:i+batch_size])
      loss,logits= outputs[:2]
      predictions = torch.argmax(logits, dim=1).flatten()
      metrics.add_batch(references=labels[i:i+batch_size], predictions=predictions)
      dict_metrics = metrics.compute()
      accuracys.append(dict_metrics['accuracy'])
      f1s.append(dict_metrics['f1'])
      precisions.append(dict_metrics['precision'])
      recalls.append(dict_metrics['recall'])
      total_loss += loss.item()
       #confusion matrix:
      matrix = np.array(tf.math.confusion_matrix(labels[i:i+batch_size],predictions))
      c_matrix = c_matrix + matrix.reshape((2,2))
    with torch.no_grad():
        avg_accuracy = np.mean(np.array(accuracys))
        avg_f1 = np.mean(np.array(f1s))
        avg_precision = np.mean(np.array(precisions))
        avg_recall = np.mean(recalls)
    return avg_accuracy, avg_f1, avg_precision, avg_recall,c_matrix

In [ ]:
# Build the 32 parameter sets
hyperparameters = []

for batch in batch_size:
    for epoch in num_epochs:
        for lr in learning_rate:
            hyperparameters.append([batch,epoch,lr])

for pair in hyperparameters:
    file = ('/content/drive/MyDrive/project/model_batch%d_epoch%d_lr{}' %(pair[0],pair[1])).format(pair[2])
    resultfile =('/content/drive/MyDrive/project/model_batch_result%d_epoch%d_lr{}.txt' %(pair[0],pair[1])).format(pair[2])
    if not os.path.isfile(file):
      model,train_accuracy,train_loss = train_model(pair[0],pair[1],pair[2])
      train_accuracys.append(train_accuracy)
      train_losses.append(train_loss)
      torch.save(model, file)
      test_accuracy,test_f1,test_precision,test_recall,test_cmatrix = test_model(model,test,pair[0])
      # We finally decided to use only accuracy and f1 score to evaluate the test phase performance
      train_result = open(resultfile,'w')
      train_result.write("%s\t%s\t%s\t%s" %(train_accuracy,train_loss,test_accuracy,test_f1))
      train_result.close()

# Since train losses and train accuracys are strongly related, so we only use the former one.